In [1]:
import pandas as pd
import os
import gc
from tqdm import tqdm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
os.getcwd()

'C:\\Users\\Kotani Lab\\Desktop\\ML_senior_project\\ML-Based-Adaptive-Cybersecurity-Incident-Detection\\Code_and_model\\Program'

In [2]:
df = pd.read_csv('kdd/train_test_folder/train_kdd/train.csv')
traindf = pd.read_csv('./kdd/dataset/mix_dataset/neptune.csv')

X_main = df.drop('label', axis=1)
y_main = df['label']

train_test_index = [df['Unnamed: 0'], df['Unnamed: 0']]


In [42]:
from multiprocessing import Pool, cpu_count

window_size = 64
batch_size = 1
epochs = 10
n_features = X_main.shape[1]

DL_args = [window_size, batch_size, epochs]

chunk_size = 200
X = traindf.drop('label', axis=1)
y = traindf['label']

Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)


In [43]:
def sequential_models(window_size, n_features):

    models = {
        'LSTM' : lstm(window_size, n_features)
    }

    return models

def lstm(window_size, n_features):
    model = Sequential()
    model.add(LSTM(512, input_shape=(window_size, n_features), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = RMSprop(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model
models = sequential_models(window_size, n_features)

In [44]:
def process_data(Data, train_index, test_index, batch_size, window_size, chunk_size=200):
    def rearrange_sequences(generator, index, batch_size, window_size):
        rearranged_data = []

        for idx in tqdm(index, desc="Processing sequences"):
            if idx >= window_size - 1:  # Ensure index has enough preceding samples for a full sequence
                sequence_end = idx + 1  # Sequence ends at the current index (inclusive)
                sequence_start = sequence_end - window_size  # Sequence starts 'window_size' samples before the end

                batch_x = generator.data[sequence_start:sequence_end]  # Extract feature sequence
                batch_y = generator.targets[idx]  # Corresponding label

                rearranged_data.append((batch_x, batch_y))

        return rearranged_data
    # Process training data
    print('Training data processing...')
    train_data = rearrange_sequences(Data, train_index, batch_size, window_size)
    gc.collect()
    
    print('Testing data processing...')
    test_data = rearrange_sequences(Data, test_index, batch_size, window_size)
    del rearrange_sequences
    gc.collect()
    return train_data, test_data
    
def separate_features_labels(data, window_size):
    features = np.array([item[0] for item in data], dtype=np.float32)  # item[0] is each sequence
    labels = np.array([item[1] for item in data], dtype=np.float32)  # item[1] is the corresponding label

    # Ensure features are reshaped to (number_of_sequences, window_size, number_of_features_per_timestep)
    features = features.reshape(-1, window_size, features.shape[-1])

    return features, labels

def training_DL(models, df, DL_args, train_test_index):
    results = {}
    X = df.drop('label', axis=1)
    y = df['label']
    
    window_size, batch_size, epochs = DL_args
    
    Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)
    print(f'Data type {type(Data)}')
    train_index, test_index = train_test_index
    print(train_index)
    print(test_index)
    print('Processing Training data...')
    
    train_data, test_data = process_data(Data, train_index, test_index, batch_size, window_size)
    gc.collect()
    X_train, y_train = separate_features_labels(train_data, window_size)
    X_test, y_test = separate_features_labels(test_data, window_size)
    return X_train, X_test, y_train, y_test

In [45]:
Data = TimeseriesGenerator(X, y, length=window_size, sampling_rate=1, batch_size=batch_size)
print(f'Data type {type(Data)}')
train_index, test_index = train_test_index
print('Processing Training data...')

X_train, X_test, y_train, y_test = training_DL(models, traindf, DL_args, train_test_index)


Data type <class 'keras.preprocessing.sequence.TimeseriesGenerator'>
Processing Training data...
Data type <class 'keras.preprocessing.sequence.TimeseriesGenerator'>
0          90597
1          89016
2         106202
3          74603
4         104247
           ...  
103956     38990
103957    122689
103958    126235
103959     10237
103960     94409
Name: Unnamed: 0, Length: 103961, dtype: int64
0          90597
1          89016
2         106202
3          74603
4         104247
           ...  
103956     38990
103957    122689
103958    126235
103959     10237
103960     94409
Name: Unnamed: 0, Length: 103961, dtype: int64
Processing Training data...
Training data processing...


Processing sequences: 100%|██████████| 103961/103961 [00:02<00:00, 39474.03it/s]


Testing data processing...


Processing sequences: 100%|██████████| 103961/103961 [00:02<00:00, 36335.76it/s]


In [46]:
# window_size = 64
# batch_size = 4
# epochs = 10
X_train[0][63]

array([0.0000000e+00, 2.5362983e-08, 9.9241383e-08, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.9138943e-03,
       1.9569471e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 5.0000000e-01, 1.0000000e+00, 0.0000000e+00,
       7.8039217e-01, 3.1372551e-02, 3.9999999e-02, 3.9999999e-02,
       9.9999998e-03, 2.5000000e-01, 9.9999998e-03, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [51]:
train_test_index[0][0]

90597

In [47]:
X_train.shape

(103925, 64, 122)

In [71]:
y_train.shape

(103925,)

In [49]:
Data[0][0]

array([[[0.00000000e+00, 3.55806412e-07, 0.00000000e+00, ...,
         0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.05799870e-07, 0.00000000e+00, ...,
         0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 2.24643560e-07, 2.13292635e-06, ...,
         0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]])

In [21]:
y_train[0]

0.0

In [39]:
y

0         0
1         0
2         1
3         0
4         0
         ..
148512    0
148513    0
148514    0
148515    0
148516    0
Name: label, Length: 148517, dtype: int64

In [66]:
sequence_start_index = train_test_index[0][1] - (window_size - 1)
sequence_start_index

88953

In [67]:
batch_index = sequence_start_index // batch_size
batch_index

88953

In [68]:
position_within_batch = sequence_start_index % batch_size
position_within_batch

0

In [69]:
# Original data point
array = traindf.drop('label', axis=1).iloc[train_test_index[0][1]].to_numpy()

# Data from the generator
datagen = Data[batch_index][0][0][window_size-1]

# Comparison
equal = np.array_equal(array, datagen)
equal